In [2]:
#This is the beginnings of making a script that can take a general dataset
#Containing data on whether an individual in a specific department has filled out the 'About me' Section
# And/Or has created an avatar
import pandas as pd
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
import networkx as nx
import csv
import json


In [3]:
data_path = r"/Users/Owner/Documents/Work_transfer/Data/GCconnex/Profile Statistics/"

In [4]:
df = pd.read_csv(data_path+'Anonymous Department Profile Stats.csv')

In [5]:
#Separating the departments and profile codes into different DataFrames
dep = df.drop('Code', 1)
df = df.drop('Department', 1)

In [6]:
# Further Splitting the codes DataFrame into two and removing duplicate entries
#If somebody edited their about me profile multiple times they will appear multiple times
df2 = df.loc[df['Code'] == 73]
df1 = df.loc[df['Code'] == 55]
df2.drop_duplicates(inplace=True)
df1.drop_duplicates(inplace=True)

C:\Users\Owner\Anaconda3\lib\site-packages\pandas\util\decorators.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [7]:
#Merging the two code dataframes into one single dataframe containing telling whether that individual
#Has filled uploaded an avatar or filled out the "about me" section
#This way each code occupies a column
df = pd.merge(df1, df2, how='outer', on='User GUID')

In [8]:
df.head()

,Code_x,User GUID,Code_y
0,55,22,73
1,55,363,NaN
2,55,224,73
3,55,460,73
4,55,70,73


In [9]:
#Merging the departments dataframe and codes dataframe
df = pd.merge(df, dep, how='outer', on='User GUID')

In [10]:
df = df.drop_duplicates()


In [11]:
df.columns = ['About me', 'User GUID', 'Avatar', 'Department']

In [12]:
df = df[['User GUID', 'Department', 'About me', 'Avatar']]

In [13]:
df

,User GUID,Department,About me,Avatar
0,22,international.gc.ca,55,73
2,363,servicecanada.gc.ca,55,NaN
3,224,ceaa-acee.gc.ca,55,73
5,460,NRCan-RNCan.gc.ca,55,73
7,70,tbs-sct.gc.ca,55,73
9,106,tpsgc-pwgsc.gc.ca,55,73
11,537,NRCan-RNCan.gc.ca,55,73
13,1110,pch.gc.ca,55,73
15,1420,pwgsc.gc.ca,55,73
17,1217,hc-sc.gc.ca,55,73


In [14]:
#Creating a column that indicates whether a user has both or none
df['Both'] = df['About me'] + df['Avatar']

In [15]:
# A series of functions that transform the codes from numbers to Y and N's.
def f(x):
    if x == 128:
        return 'Y'
    else:
        return 'N'

def g(x):
    if x == 55:
        return 'Y'
    else:
        return 'N'
    
def h(x):
    if x == 73:
        return 'Y'
    else:
        return 'N'

df['Both'] = df['Both'].apply(f)
df['About me'] = df['About me'].apply(g)
df['Avatar'] = df['Avatar'].apply(h)


In [16]:
#Exporting the raw data for verification that the code did its job and just in case this is useful
df.to_csv(data_path+'Department Profile Stats Raw.csv')

In [17]:
depnum = df
depnum['Department'] = depnum['Department'].str.lower()

In [18]:
dept = depnum.Department

In [19]:
dept_sort = set(dept)

In [20]:
dept_dict = {}


In [21]:
with open(os.path.join(data_path, "csv_keys.csv"), "r") as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)
    
    for row in reader:
        email, acronym = row
        dept_dict[email] = acronym

In [22]:
dept_dict['cadets.gc.ca'] = 'CADETS'
dept_dict['canada.gc.ca'] = 'CANADA'
dept_dict['canada.ca'] = 'CANADA'
dept_dict['tribunal.gc.ca'] = 'TRIBUNAL'
dept_dict['cannor.gc.ca'] = 'CED/DEC'
dept_dict['ci-oic.gc.ca'] = 'CI/OIC'
dept_dict['ccgs-ngcc.gc.ca'] = 'CCGS/NGCC'
dept_dict['god.ccgs-ngcc.gc.ca'] = 'CCGS/NGCC'
dept_dict['clo-ocol.gc.ca'] = 'OCOL/CLO'
dept_dict['csps.gc.ca'] = 'CSPS/EFPC'
dept_dict['interenational.gc.ca'] = 'DFAITD/MAECD'
dept_dict['cnb-ncw.gc.ca'] = 'CNB/NCW'
dept_dict['ncw-cnb.gc.ca'] = 'CNB/NCW'
dept_dict['nfb.gc.ca'] = 'NFB/ONF'
dept_dict['nrccan-rncan.gc.ca'] = 'NRCAN/RNCAN'
dept_dict['nserc-crsng.gc.ca'] = 'NSERC/CRSNG'
dept_dict['pbc-clcc.gc.ca'] = 'PBC/CLCC'
dept_dict['pco.bcp.gc.ca'] = 'PCO/BCP'
dept_dict['pipsc.ca'] = 'PIPSC/IPFPC'
dept_dict['ps.sp.gc.ca'] = 'PS/SP'
dept_dict['servicecanada.gc.ca.gc.ca'] = 'HRSDC/RHDSC'






In [23]:
dep = pd.DataFrame.from_dict(dept_dict, orient='index')

In [24]:
depnum = depnum.replace({'Department': dept_dict})

In [25]:
depstats = depnum.groupby('Department').count()


In [26]:
depabout = depnum[(depnum['About me'] == 'Y')]

In [27]:
depabout = depabout.groupby('Department').count()

In [28]:
deppic = depnum[(depnum['Avatar'] == 'Y')]
deppic = deppic.groupby('Department').count()

In [29]:
depboth = depnum[(depnum['Both'] == 'Y')]
depboth = depboth.groupby('Department').count()

In [30]:
depagg = pd.concat([depstats['User GUID'], depabout['About me'], deppic['Avatar'], depboth['Both']], axis=1, keys= ['Users', 'About me', 'Avatar', 'Both'])

In [31]:
depagg = depagg.fillna(0)

In [32]:
depusers = pd.read_csv(data_path+'Department Users CSV.csv')
depusers['Department'] = depusers['Department'].str.lower()
depusers = depusers.replace({'Department': dept_dict})
depusers['Count'] = np.arange(len(depusers))
depusers = depusers.groupby('Department').count()

In [33]:
depagg['Total Users'] = depusers['Count']

In [34]:
depagg.to_csv(data_path+'Department Profile Statistics.csv')

In [35]:
depperc = depagg
depperc['Users'] = depperc['Users']/depperc['Total Users']
depperc['About me'] = depperc['About me']/depperc['Total Users']
depperc['Avatar'] = depperc['Avatar']/depperc['Total Users']
depperc['Both'] = depperc['Both']/depperc['Total Users']


In [36]:
depperc.to_csv(data_path+'Department Profile Statistics Percentages.csv')

NameError: name 'deppagg' is not defined